In [11]:
# Import packages
import pandas as pd
import numpy as np

In [2]:
train=pd.read_csv('/Users/bhanu/train.csv',parse_dates = ['date_time'],nrows=10000)
test=pd.read_csv('/Users/bhanu/test.csv',nrows=10000)

In [6]:
hotels_by_dest=train.groupby(['srch_destination_id','hotel_cluster'])['is_booking'].agg(['sum','count'])
#hotels_by_dest.columns=('bookings','total')
hotels_by_dest.reset_index(inplace=True)
hotels_by_dest.head(10)

,srch_destination_id,hotel_cluster,sum,count
0,11,94,1,2
1,16,7,1,2
2,19,20,0,1
3,19,30,0,1
4,19,40,0,4
5,19,61,0,1
6,56,57,0,1
7,66,50,0,1
8,68,12,0,2
9,68,30,0,2


In [8]:
CLICK_WEIGHT = 0.05
agg = hotels_by_dest.groupby(['srch_destination_id','hotel_cluster']).sum().reset_index()
agg['count'] -= agg['sum']
agg = agg.rename(columns={'sum':'bookings','count':'clicks'})
agg['popularity'] = agg['bookings'] + CLICK_WEIGHT * agg['clicks']
agg.head(20)

,srch_destination_id,hotel_cluster,bookings,clicks,popularity
0,11,94,1,1,1.05
1,16,7,1,1,1.05
2,19,20,0,1,0.05
3,19,30,0,1,0.05
4,19,40,0,4,0.20
5,19,61,0,1,0.05
6,56,57,0,1,0.05
7,66,50,0,1,0.05
8,68,12,0,2,0.10
9,68,30,0,2,0.10


In [9]:
def most_popular(group, n_max=5):
    relevance = group['popularity'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets

In [12]:
most_pop = agg.groupby(['srch_destination_id']).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
most_pop.head()

,hotel_cluster
srch_destination_id,
11,94
16,7
19,40 61 30 20
56,57
66,50


In [13]:
test = test.merge(most_pop, how='left',left_on='srch_destination_id',right_index=True)
test.head()

,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,0,2015-09-03 17:09:54,2,3,66,174,37449,5539.0567,1,1,...,2016-05-23,2,0,1,12243,6,6,204,27,NaN
1,1,2015-09-24 17:38:35,2,3,66,174,37449,5873.2923,1,1,...,2016-05-15,2,0,1,14474,7,6,204,1540,NaN
2,2,2015-06-07 15:53:02,2,3,66,142,17440,3975.9776,20,0,...,2015-07-27,4,0,1,11353,1,2,50,699,21 91 84 59 48
3,3,2015-09-14 14:49:10,2,3,66,258,34156,1508.5975,28,0,...,2015-09-16,2,0,1,8250,1,2,50,628,1 45 54 88 24
4,4,2015-07-17 09:32:04,2,3,66,467,36345,66.7913,50,0,...,2015-07-23,2,0,1,11812,1,2,50,538,47 16
